#### Refactor : speed up data download

> 1. 기존 데이터 다운로드 패키지의 속도는 다음과 같다.

![테스트결과](C:/Users/pc/JupyterWorkflowEx/img/downloadtime.png)

> 2. 속도에 영향을 주는 부분을 찾아 refactoring 한다.
>> * 'data.py'의 속도를 잡아먹는 부분(데이터를 가져와 parsing하는 부분)을 가져와 변경한다.

In [2]:
from jupyterworkflow.data import get_fremont_data
import pandas as pd

def test_fremont_data():
    data = get_fremont_data()
    #data.columns를 실행하면 인덱스, 칼럼명, 타입이 표시 되므로 실행 후 칼럼명을 카피하면 좋음
    # all(data.columns == ['West', 'East', 'Total']) : 칼럼명이 일치하면 True
    assert all(data.columns == ['West', 'East', 'Total'])
    assert isinstance(data.index, pd.DatetimeIndex)

In [3]:
test_fremont_data() # run 'test_fremont_data' function

In [9]:
# 'data.py'에서 가져 온 부분을 변경
# data = pd.read_csv('Fremont.csv', index_col = 'Date', parse_dates = True)
# data.columns = ['West', 'East']
# data['Total'] = data['West'] + data['East']

data = pd.read_csv('Fremont.csv', index_col = 'Date')
# data.head() : 데이터가 로드 된 것 확인
# data.index.dtype : 데이터 타입을 확인
# pd.to_datetime(data.index) #날짜, 시간 형식의 표식 포맷을 확인 후 
# 'http://strftime.org/'의 포맷 형식을 따라 변경

try:
    data.index = pd.to_datetime(data.index, format = '%m/%d/%Y %H:%M:%S %p') # 이 문장이 pd.to_datetime(data.index)보다 훨씬 빨리 로드 됨
except TypeError:
    data.index = pd.to_datetime(data.index)

# 'data.py'에 try ~ except 부분을 추가하여 변경

DatetimeIndex(['2012-10-03 12:00:00', '2012-10-03 01:00:00',
               '2012-10-03 02:00:00', '2012-10-03 03:00:00',
               '2012-10-03 04:00:00', '2012-10-03 05:00:00',
               '2012-10-03 06:00:00', '2012-10-03 07:00:00',
               '2012-10-03 08:00:00', '2012-10-03 09:00:00',
               ...
               '2018-04-30 02:00:00', '2018-04-30 03:00:00',
               '2018-04-30 04:00:00', '2018-04-30 05:00:00',
               '2018-04-30 06:00:00', '2018-04-30 07:00:00',
               '2018-04-30 08:00:00', '2018-04-30 09:00:00',
               '2018-04-30 10:00:00', '2018-04-30 11:00:00'],
              dtype='datetime64[ns]', name='Date', length=48864, freq=None)

In [11]:
data.columns = ['West', 'East']
data['Total'] = data['West'] + data['East']

# 속도가 1.81 seconds로 개선됨